In [149]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pandas as pd

# 6. Preprocess Data and Create Labels and Features

In [150]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [151]:
df = pd.read_csv(f"https://docs.google.com/spreadsheets/d/1FcGVkGoaUH1eb0cs9Ip3Y2tfxv7a2GfEzXV7FMk0vV8/export?format=csv")

actions =  np.array(list(df["actions"]))
label_map = {label:num for num, label in enumerate(actions)}

DATA_PATH = os.path.join('StoreData') 
# DATA_PATH = os.path.join('MP_Data') 
# Videos are going to be 30 frames in length
sequence_length = 60

In [152]:
actions.shape[0]

3

In [153]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [154]:
sequences

[[array([0., 0., 0., ..., 0., 0., 0.]),
  array([ 0.46062177,  0.35453999, -0.54785973, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.46093756,  0.35239086, -0.67321724, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.46090156,  0.35176641, -0.67162961, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.46081191,  0.35147864, -0.65736377, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.4604857 ,  0.35131204, -0.67628044, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.45948696,  0.35164589, -0.72588074, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.45877567,  0.35189083, -0.76731527, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.45786047,  0.35199347, -0.77949989, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.45779395,  0.35225496, -0.76171899, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.45777506,  0.35239121

In [155]:
np.array(sequences).shape

(90, 60, 1662)

In [156]:
np.array(labels).shape

(90,)

In [157]:
X = np.array(sequences)

In [158]:
X.shape

(90, 60, 1662)

In [159]:
y = to_categorical(labels).astype(int)

In [160]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [161]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [163]:
# y_test.shape
X_train

array([[[ 0.41193324,  0.35385552, -0.72947735, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.41192919,  0.354083  , -0.71197969, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.41214958,  0.35453957, -0.71786201, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.40714693,  0.34928235, -0.74281257, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.40728897,  0.35024786, -0.79147369, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.40741783,  0.35042781, -0.75759494, ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.37621692,  0.35548407, -0.62450987, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.38090637,  0.36096001, -0.76479203, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.39143243,  0.36406583, -0.7414276 , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.37821847,  0.38865545, -0.68489116, ...,  

# 7. Build and Train LSTM Neural Network

In [164]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [166]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [167]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(60,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [168]:
res = [0.7,0.2,0.1]

In [169]:
res

[0.7, 0.2, 0.1]

In [170]:
actions[np.argmax(res)]

'hello'

In [171]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [172]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 4s 192ms/step - loss: 9.7073 - categorical_accuracy: 0.3647
Epoch 2/2000
3/3 [==============================] - 1s 176ms/step - loss: 53.4359 - categorical_accuracy: 0.4000
Epoch 3/2000
3/3 [==============================] - 1s 191ms/step - loss: 1.3673 - categorical_accuracy: 0.2588
Epoch 4/2000
3/3 [==============================] - 1s 208ms/step - loss: 9.1412 - categorical_accuracy: 0.3412
Epoch 5/2000
3/3 [==============================] - 1s 204ms/step - loss: 19.9142 - categorical_accuracy: 0.3294
Epoch 6/2000
3/3 [==============================] - 1s 191ms/step - loss: 136.3150 - categorical_accuracy: 0.3647
Epoch 7/2000
3/3 [==============================] - 1s 166ms/step - loss: 54.6247 - categorical_accuracy: 0.3647
Epoch 8/2000
3/3 [==============================] - 1s 189ms/step - loss: 106.8752 - categorical_accuracy: 0.3294
Epoch 9/2000
3/3 [==============================] - 1s 191ms/step - loss: 201.5911 - categorical_

KeyboardInterrupt: 

#real TIme

In [174]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [175]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [176]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [177]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [178]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [179]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [182]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-60:]
        
        if len(sequence) == 60:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(np.argmax(res))
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord(' '):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [181]:
cap.release()
cv2.destroyAllWindows()